In [ ]:
import numpy as np
import os

import rasterio.mask

import fiona
import pyproj
from shapely.geometry import shape
from shapely.ops import transform

wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:32630')

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform

with fiona.open("larioja/anguiano.geojson", "r") as bbox_geojson:
    shapes = [transform(project, shape(feature["geometry"])) for feature in bbox_geojson]

print(shapes[0].wkt)

# NDVI (Sentinel 2) = (B8 – B4) / (B8 + B4)
# B4	10 m	665 nm	Red
# B8	10 m	842 nm	Visible and Near Infrared (VNIR)

bands = './before/bands'

def create_mask(image: str):

    with rasterio.open(image) as src:
        try:
            mask_image, mask_transform = rasterio.mask.mask(src,
                                                            shapes,
                                                            crop=True,
                                                            all_touched=True,
                                                            nodata=src.nodata)
        except ValueError:
            print(f'Image {image} does not intersect with the bounding box')
            return None, None

        out_meta = src.meta

        out_meta.update({"driver": "GTiff",
                         "height": mask_image.shape[1],
                         "width": mask_image.shape[2],
                         "transform": mask_transform,
                         "nodata": src.nodata
                         })

        filename = f"{image}.masked.tif"
        with rasterio.open(filename, "w", **out_meta) as dest:
            dest.write(mask_image)
            mask_profile = dest.profile.copy()

            return mask_image, mask_profile

# walk over a directory of Sentinel 2 images
for root, dirs, files in os.walk(bands):
    for file in files:
        if file.endswith('.jp2'):
            if "B08" in file:
                b8 = os.path.join(root, file)
                b4 = os.path.join(root, file.replace('B08', 'B04'))

                b8_array, b8_profile = create_mask(b8)
                b4_array, b4_profile = create_mask(b4)

                BAND = 1
                B8 = 0
                B4 = 1

                if b8_array is None or b4_array is None:
                    continue

                sentinel_b8 = np.array(b8_array, dtype=b8_array.dtype).astype(float)
                sentinel_b4 = np.array(b4_array, dtype=b4_array.dtype).astype(float)

                np.seterr(divide='ignore', invalid='ignore')
                ndvi = (sentinel_b8 - sentinel_b4) / (sentinel_b8 + sentinel_b4)

                ndvi[np.isnan(ndvi)] = 0

                print('Max NDVI: {m}'.format(m=ndvi.max()))
                print('Mean NDVI: {m}'.format(m=ndvi.mean()))
                print('Median NDVI: {m}'.format(m=np.median(ndvi)))
                print('Min NDVI: {m}'.format(m=ndvi.min()))

                b8_profile.update({
                    'dtype': 'float32',
                    'nodata': 0,
                    'height': ndvi.shape[1],
                    'width': ndvi.shape[2],
                })

                filename = f"{b8.replace('B08', 'NDVI')}.tif"
                print(f'Writing {filename}')
                with rasterio.open(filename, 'w', **b8_profile) as dst:
                    dst.write(ndvi)


